# Explore

#### 1) Which county has the highest mean tax_value?  Is county related to tax_value?

#### 2) Does total_sqft relate to tax_value?


#### 3) Does number of bedrooms relate to tax_value?


#### 4) Does zip code relate to tax_value?


#### 5) Does year_built relate to tax_value?